# Training
Zero-shot, one-shot and few-shot training using the quantized model of Llama 3 (8B). Install llama_cpp_python package with CUDA to make sure GPU is deployed.

In [ ]:
from llama_cpp import Llama
llm=Llama(
    model_path=r".\Model\Meta-Llama-3-8B.Q2_K.gguf",  ###Download from Hugging face https://huggingface.co/QuantFactory/Meta-Llama-3-8B-GGUF
    n_ctx=2048,n_gpu_layers=-1)

In [ ]:
# define Llama-function for the binary classification task
def LlamaBin(sentence, examples_list):
    temperature=0
    top_p=0.7
    echo=False
    max_tokens=len(sentence) + 1000
    


    #sentence= five_folds['sentence'].iloc[2]

    if len(examples_list) > 0:
        examples_string = "# Voorbeelden\n"
        for i in range(len(examples_list)):
            example = examples_list[i]
            string_part, answer = example[0], example[1]
            examples_string += f"Patiënt {i+1}: {string_part}\n"
            examples_string += f"Antwoord {i+1}: {answer}\n"
    elif len(examples_list) == 0:
        examples_string = "empty"

    if len(sentence) > 500:
        sentence = sentence[:500]
    else:
        pass

    prompt = f"""
    # Vraag
    Jij bent een binaire classficatie model. Geef aan voor de volgende zinnen met patiëntinformatie of deze 
    een beschrijving van een WHO Performance Status bevatten. 
    Een WHO Performance Status beschrijft de mate waarin een patiënt in staat is zelf activiteiten te ondernemen. 
    Geef antwoord '0' wanneer de patiëntinformatie geen Performance Status bevat. Geef antwoord '1' wanneer de 
    patiëntinformatie wél een Performance Status bevat.
    Zinnen met Performance Status beschrijven een patient zoals:
    * 1: Volledig actief
    * 1: Beperkt in fysiek inspannende activiteit, bijv. Licht huiswerk, kantoorwerk.
    * 1: Ambulant en geschikt voor alle zelfzorg, maar niet in staat om enige werkactiviteiten uit te voeren.
    * 1: Beperkt tot bed of meer dan 50% wekelijkse uren.
    * 1: Heel beperkt tot bed of stoel.
    * 1: Dood
    Zinnen zonder Performance Status bevatten:
    * 0: resultaten van klinische examen
    * 0: therapie
    * 0: voeding
    * 0: kanker diagnose
    * 0: andere informatie rondom patient
    
    #Toegestaan antwoorden
    Antwoord|'0','1'
    
    {examples_string}
    
    # Patiëntinformatie 
    {sentence}

    # Antwoord
    Antwoord:"""
    ## Question
    #Act as a binary classification model. Your task is to classify the input according to the following instructions. Return 0 as Output if the input doesn't contain 
    #a WHO Performance Status or return 1 as Output if the input contains a WHO Performance Status. A WHO Performance Status
    #describes how a patient is able to perform daily activities independently, for example taking care of theirselves, 
    #household activities, sport, work and physical movement. The input is in Dutch language and contains a sentence extracted from
    #patient clinical notes.
         
    #{examples_string}

    ## Input
    #{sentence}

    ## Output
    #Output: """

    model_output=llm(
                    prompt,
                    max_tokens=max_tokens,
                    temperature=temperature,
                    echo=False, #,
                    stop=['# Vraag', '\n']
                    #max_new_tokens=3
                    #n_predict=3
                    #stopping_criteria=StoppingCriteria(['#STOP'])
                    #stream=True
                    )


    return model_output #['choices'][0]['text'].split(": ")[-1][0]


def extract_digit(string):
    match = re.search(r'\d', string)
    if match:
        return int(match.group())
    else:
        return np.nan


def perform_llama_binary(examples_list, five_folds):
    #print(examples_list)
    new_labels = []
    i=0
    for sentence in five_folds['sentence']:#.iloc[0:50]:
        print(i)
        i=i+1
        if len(sentence)>10:
       
            new_label = LlamaBin(sentence, examples_list)

            try:
                #print(new_label)
                print('Answer is ' + new_label['choices'][0]['text'][1])
            except:
                new_label = LlamaBin(sentence, examples_list)

            new_label = new_label['choices'][0]['text'][1]
            if (new_label=='0' or new_label=='1')==False:
                new_label = LlamaBin(sentence, examples_list)
                print('2nd tentative: Answer is ' + new_label['choices'][0]['text'][1])
                new_label = new_label['choices'][0]['text'][1]

            #except:
             #   print(new_label)
             #   print(new_label['choices'][0]['text'])
             #   new_label = new_label['choices'][0]['text']
        else:
            new_label=np.nan
    
        new_labels.append(new_label)
    return new_labels
    


In [ ]:
# define Llama-function for the regression task
def LlamaClasses(sentence, examples_list):

    #echo=True
    max_tokens=len(sentence) + 1000
    
    if len(examples_list) > 0:
        examples_string = "# Voorbeelden\n"
        for i in range(len(examples_list)):
            example = examples_list[i]
            string_part, answer = example[0], example[1]
            examples_string += f"Patiënt {i+1}: {string_part}\n"
            examples_string += f"Antwoord {i+1}: {answer}\n"
    elif len(examples_list) == 0:
        examples_string = "empty"
    
    if len(sentence) > 1000:
        sentence = sentence[:1000]
    else:
        pass
    
    prompt = f"""
    # Vraag
    Jij bent een regressie model. Geef een WHO Performance Status score van een patiënt. De waardes zijn als volgt: 
    * 0: Volledig actief, in staat om alle voorziekte prestaties zonder beperking uit te voeren.
    * 1: Beperkt in fysiek inspannende activiteit, maar ambulant en in staat om werk van lichte of sedentaire aard te verrichten, bijv. Licht huiswerk, kantoorwerk.
    * 2: Ambulant en geschikt voor alle zelfzorg, maar niet in staat om enige werkactiviteiten uit te voeren. Tot en met meer dan 50% van de wekelijkse uren.
    * 3: Alleen beperkte zelfverzorging, beperkt tot bed of meer dan 50% wekelijkse uren.
    * 4: Kan geen zelfzorg uitvoeren. Heel beperkt tot bed of stoel.
    * 5: Patient overleden
    
    {examples_string}
    
    # Patiëntinformatie 
    {sentence}

    # Antwoord
    Antwoord:"""

    #print(prompt)

#    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
#    with torch.no_grad():
#        outputs = model.generate(**inputs, max_length = 5000, eos_token_id=tokenizer.encode('\n'))

    #print(tokenizer.decode(outputs[0], skip_special_tokens=True))

#    output = tokenizer.decode(outputs[0], skip_special_tokens=True)
#    new_label = output.split(": ")[-1]


    temperature=0
    top_p=0.7
    




    model_output=llm(
                    prompt,
                    max_tokens=max_tokens,
                    temperature=temperature,
                    echo=False,
                    stop=['# Vraag', '\n']
                    #stream=True
                    )
    return model_output


import re
import numpy as np

def extract_digit(string):
    match = re.search(r'\d', string)
    if match:
        return int(match.group())
    else:
        return np.nan


def perform_llama_classes(examples_list, five_folds_classes, ex_df):
    #print(examples_list)
    new_labels = []
    i=0

    for sentence in five_folds_classes['sentence']:
        print(i)
        i=i+1
        if sentence in ex_df:
            new_label=np.nan
        else:

            new_label = LlamaClasses(sentence, examples_list)
            print(new_label)
            try:
                new_label=new_label['choices'][0]['text'][1]
            except:
                new_label=np.nan

            
        #new_label = extract_digit(new_label)
        print(new_label)
        new_labels.append(new_label)
    return new_labels
    

## Validation set

In [ ]:
df_bin_examples = pd.read_csv(r'./Intermediate results/df_bin_examples_acsess.csv', sep= '\t')
df_class_examples = pd.read_csv(r'./Intermediate results/df_class_examples_acsess.csv', sep= '\t')
folds = pd.read_csv(r'./Intermediate results/val_folds_validated.csv', sep= '\t')
five_folds=folds[folds.relevance_manual==folds.validated]

# define results dataset
acsess_bin_results = five_folds[['note_nr', 'sentence',
       'manual_sentence_labels', 'relevance_manual', 'round', 'annotator',
       'set', 'fold']]

### Binary classification task
The objective is to detect if there is a Performance status description or not.

In [ ]:
k=0 #zero-shot
#k=1 #one-shot
#k=5 #few-shot with 5 examples

for examples_num in range(k): 
    print("K=",examples_num)
    if examples_num > 0:
        examples = df_bin_examples[df_bin_examples['k'] == examples_num]
        examples_list = []
        for index, row in examples.iterrows():
            examples_list.append([row['example'], row['label']])
    elif examples_num == 0:
        examples_list = []
    
    new_labels = perform_llama_binary(examples_list, five_folds)
    acsess_bin_results[f"binary k={examples_num}"] = new_labels
    plt.hist(acsess_bin_results[f'binary k={examples_num}'].dropna())

In [ ]:
#Consistency check in case of hallucinations
for k in range(k):
    for idx,row in acsess_bin_results.iterrows():
        if (row[f'binary k={k}']=='0' or row[f'binary k={k}']=='1' or row[f'binary k={k}']==0 or row[f'binary k={k}']==1)==False:
            acsess_bin_results.loc[idx,f'binary k={k}']=np.nan


In [ ]:
#Save results
acsess_bin_results.to_csv(r'./Results/acsess_bin_results_val.csv', sep='\t')

### Regression task
The objective is to assign the score from 0 to 5 to sentences containing a Performance Status description.

In [ ]:
# define results dataset
acsess_class_results = five_folds[['note_nr', 'sentence',
       'manual_sentence_labels', 'relevance_manual', 'round', 'annotator',
       'set', 'fold']]


acsess_class_results = acsess_class_results[five_folds['relevance_manual'] == 1]
acsess_class_results = acsess_class_results.reset_index(drop=True)

five_folds_classes = five_folds[five_folds['relevance_manual'] == 1]
five_folds_classes = five_folds_classes.reset_index(drop=True)

In [ ]:
k=0 #zero-shot
#k=1 #one-shot
#k=5 #few-shot with 5 examples

for examples_num in range(k): 
    print("K=",examples_num)
    examples = df_class_examples[df_class_examples['k'] == examples_num]
    examples_list = []
    for index, row in examples.iterrows():
        examples_list.append([row['example'], row['label']])
    
    new_labels = perform_llama_classes(examples_list, five_folds_classes,df_class_examples[df_class_examples['k'] == examples_num].example.values)
    acsess_class_results[f"binary k={examples_num}"] = new_labels

In [ ]:
#Consistency check in case of hallucinations
for k in range(k):
    for idx,row in acsess_class_results.iterrows():
        if (row[f'binary k={k}']=='0' or row[f'binary k={k}']=='1' or row[f'binary k={k}']=='2' or row[f'binary k={k}']=='3' or row[f'binary k={k}']=='4' or row[f'binary k={k}']=='5')==False:
            acsess_class_results.loc[idx,f'binary k={k}']=np.nan


In [ ]:
acsess_class_results.to_csv(r'./Results/acsess_class_results_val.csv', sep='\t')

## Test set

In [ ]:
df_bin_examples = pd.read_csv(r'./Intermediate results/df_bin_examples_acsess.csv', sep= '\t')
df_class_examples = pd.read_csv(r'./Intermediate results/df_class_examples_acsess.csv', sep= '\t')
folds = pd.read_csv(r'./Intermediate results/test_folds_validated.csv', sep= '\t')    
five_folds=folds[folds.relevance_manual==folds.validated]

# define results dataset
acsess_bin_results = five_folds[['note_nr', 'sentence',
       'manual_sentence_labels', 'relevance_manual', 'round', 'annotator',
       'set', 'fold']]

### Binary classification task
The objective is to detect if there is a Performance status description or not.

In [ ]:
k=0 #zero-shot
#k=1 #one-shot
#k=5 #few-shot with 5 examples

for examples_num in range(k): 
    print("K=",examples_num)
    if examples_num > 0:
        examples = df_bin_examples[df_bin_examples['k'] == examples_num]
        examples_list = []
        for index, row in examples.iterrows():
            examples_list.append([row['example'], row['label']])
    elif examples_num == 0:
        examples_list = []
    
    new_labels = perform_llama_binary(examples_list, five_folds)
    acsess_bin_results[f"binary k={examples_num}"] = new_labels
    plt.hist(acsess_bin_results[f'binary k={examples_num}'].dropna())

In [ ]:
#Consistency check in case of hallucinations
for k in range(k):
    for idx,row in acsess_bin_results.iterrows():
        if (row[f'binary k={k}']=='0' or row[f'binary k={k}']=='1' or row[f'binary k={k}']==0 or row[f'binary k={k}']==1)==False:
            acsess_bin_results.loc[idx,f'binary k={k}']=np.nan

In [ ]:
#Save results
acsess_bin_results.to_csv(r'./Results/acsess_bin_results_val.csv', sep='\t')

### Regression task
The objective is to assign the score from 0 to 5 to sentences containing a Performance Status description.

In [ ]:
# define results dataset
acsess_class_results = five_folds[['note_nr', 'sentence',
       'manual_sentence_labels', 'relevance_manual', 'round', 'annotator',
       'set', 'fold']]


acsess_class_results = acsess_class_results[five_folds['relevance_manual'] == 1]
acsess_class_results = acsess_class_results.reset_index(drop=True)

five_folds_classes = five_folds[five_folds['relevance_manual'] == 1]
five_folds_classes = five_folds_classes.reset_index(drop=True)

In [ ]:
k=0 #zero-shot
#k=1 #one-shot
#k=5 #few-shot with 5 examples

for examples_num in range(k): 
    print("K=",examples_num)
    examples = df_class_examples[df_class_examples['k'] == examples_num]
    examples_list = []
    for index, row in examples.iterrows():
        examples_list.append([row['example'], row['label']])
    
    new_labels = perform_llama_classes(examples_list, five_folds_classes,df_class_examples[df_class_examples['k'] == examples_num].example.values)
    acsess_class_results[f"binary k={examples_num}"] = new_labels

In [ ]:
#Consistency check in case of hallucinations
for k in range(k):
    for idx,row in acsess_class_results.iterrows():
        if (row[f'binary k={k}']=='0' or row[f'binary k={k}']=='1' or row[f'binary k={k}']=='2' or row[f'binary k={k}']=='3' or row[f'binary k={k}']=='4' or row[f'binary k={k}']=='5')==False:
            acsess_class_results.loc[idx,f'binary k={k}']=np.nan

In [ ]:
#Save results to csv
acsess_class_results.to_csv(r'./Results/acsess_class_results_test.csv', sep='\t')